In [8]:
# Import needed libraries.

from astroplan import Observer, FixedTarget
from astropy.time import Time
from astropy.table import Table, join
import astropy.table as table
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroplan import (AirmassConstraint, AtNightConstraint, 
                       TimeConstraint, MoonIlluminationConstraint, 
                       MoonSeparationConstraint)
from astroplan import is_observable
import numpy as np
from astroplan import observability_table

In [9]:
# Set observer location.

dao = Observer.at_site("Dominion Astrophysical Observatory")

In [10]:
# Set targets.

targets_raw = Table.read('Galaxies.csv', format = 'ascii')
targets_raw = table.unique(targets_raw, keys = "main_id")
targets_raw.remove_columns(["flux", "otype"])
targets = [FixedTarget(coord = SkyCoord(ra = ra*u.deg, dec = dec*u.deg), name = main_id)
           for main_id, ra, dec in targets_raw]
targets_raw.rename_column("main_id", "target name")

In [11]:
# Set constraints on time of observation, airmass, degree of twilight,
# moon illumination, and moon separation.

time_start = Time(['2019-5-1 00:00:00'])
time_end = Time(['2019-5-15 00:00:00'])
constraints = [AirmassConstraint(1.5), AtNightConstraint.twilight_astronomical(), 
               TimeConstraint(time_start, time_end), 
               MoonIlluminationConstraint(min = None, max = 0.1),
               MoonSeparationConstraint(min = 40*u.deg, max = None)]

# Check observability.
# Constructs the observability table, then removes all objects that are never observable.

observability = observability_table(constraints, dao, targets, time_range = (time_start, time_end))
observability = observability.group_by("ever observable")
observability = observability.groups[1]

In [12]:
# Creates and displays the final target data, including name,
# ra, dec, and fraction of time observable.

observability.remove_columns(["ever observable", "always observable"])
final_targets = join(targets_raw, observability)
final_targets[:10:]

target name,ra,dec,fraction of time observable
str37,float64,float64,float64
M 51,202.469575,47.19525833333333,0.12797619047619047
M 63,198.9553077198746,42.0293683550047,0.12797619047619047
M 94,192.72145,41.120152777777776,0.12797619047619047
M 101,210.80242916666668,54.34875,0.12797619047619047
M 102,226.62317083333332,55.76330833333333,0.12797619047619047
M 106,184.7400833333333,47.30371944444444,0.12797619047619047
M 108,167.87902916666664,55.67412222222222,0.11160714285714286
M 109,179.3999333333333,53.374519444444445,0.12797619047619047
NDWFS J142922.8+351219,217.34524999999994,35.20525000000001,0.12797619047619047


In [13]:
# We got rained out! Repeat observability check for one month later.

time_start = Time(['2019-6-1 00:00:00'])
time_end = Time(['2019-6-15 00:00:00'])
constraints = [AirmassConstraint(1.5), AtNightConstraint.twilight_astronomical(), 
               TimeConstraint(time_start, time_end), 
               MoonIlluminationConstraint(min = None, max = 0.1),
               MoonSeparationConstraint(min = 40*u.deg, max = None)]

observability = observability_table(constraints, dao, targets, time_range = (time_start, time_end))
observability = observability.group_by("ever observable")
observability = observability.groups[1]

In [14]:
observability.remove_columns(["ever observable", "always observable"])
final_targets = join(targets_raw, observability)
final_targets[:10:]

target name,ra,dec,fraction of time observable
str37,float64,float64,float64
M 51,202.469575,47.19525833333333,0.041666666666666664
M 63,198.9553077198746,42.0293683550047,0.041666666666666664
M 94,192.72145,41.120152777777776,0.03571428571428571
M 101,210.80242916666668,54.34875,0.041666666666666664
M 102,226.62317083333332,55.76330833333333,0.041666666666666664
M 106,184.7400833333333,47.30371944444444,0.03125
M 108,167.87902916666664,55.67412222222222,0.017857142857142856
M 109,179.3999333333333,53.374519444444445,0.03125
NDWFS J142922.8+351219,217.34524999999994,35.20525000000001,0.041666666666666664
